In [2]:
import numpy as np
import os
import csv

# ============================================================
# 1. LSTM CLASS (WITH BPTT)
# ============================================================
class LSTM:
    def __init__(self, input_size, hidden_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.total_input = input_size + hidden_size
        self.lr = learning_rate

        # States
        self.h = np.zeros(hidden_size)
        self.c = np.zeros(hidden_size)

        # Weights
        self.Wf = np.random.randn(hidden_size, self.total_input) * 0.1
        self.Wi = np.random.randn(hidden_size, self.total_input) * 0.1
        self.Wc = np.random.randn(hidden_size, self.total_input) * 0.1
        self.Wo = np.random.randn(hidden_size, self.total_input) * 0.1

        self.Wy = np.random.randn(input_size, hidden_size) * 0.1
        self.by = np.zeros(input_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def dsigmoid(self, y):
        return y * (1 - y)

    def tanh(self, x):
        return np.tanh(x)

    def dtanh(self, y):
        return 1 - y ** 2

    # ----------------------------
    # FORWARD PASS
    # ----------------------------
    def forward(self, x):
        self.x = x
        self.z = np.concatenate([self.h, x])

        self.f = self.sigmoid(np.dot(self.Wf, self.z))
        self.i = self.sigmoid(np.dot(self.Wi, self.z))
        self.c_hat = self.tanh(np.dot(self.Wc, self.z))
        self.o = self.sigmoid(np.dot(self.Wo, self.z))

        self.c_prev = self.c.copy()
        self.c = self.f * self.c + self.i * self.c_hat
        self.h = self.o * self.tanh(self.c)

        y_pred = np.dot(self.Wy, self.h) + self.by
        return y_pred

    # ----------------------------
    # BACKPROP (1-STEP BPTT)
    # ----------------------------
    def backward(self, y_pred, y_true):
        dy = y_pred - y_true  # d(MSE)/dy

        dWy = np.outer(dy, self.h)
        dby = dy

        dh = np.dot(self.Wy.T, dy)
        do = dh * self.tanh(self.c)
        do *= self.dsigmoid(self.o)

        dc = dh * self.o * self.dtanh(self.tanh(self.c))
        dc += self.f * 0  # truncated BPTT

        df = dc * self.c_prev
        df *= self.dsigmoid(self.f)

        di = dc * self.c_hat
        di *= self.dsigmoid(self.i)

        dc_hat = dc * self.i
        dc_hat *= self.dtanh(self.c_hat)

        dWf = np.outer(df, self.z)
        dWi = np.outer(di, self.z)
        dWc = np.outer(dc_hat, self.z)
        dWo = np.outer(do, self.z)

        # ----------------------------
        # WEIGHT UPDATE
        # ----------------------------
        self.Wy -= self.lr * dWy
        self.by -= self.lr * dby
        self.Wf -= self.lr * dWf
        self.Wi -= self.lr * dWi
        self.Wc -= self.lr * dWc
        self.Wo -= self.lr * dWo


# ============================================================
# 2. LOAD & NORMALIZE DATA
# ============================================================
def load_accelerometer_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError("accelerometer.csv not found")

    data = []
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            try:
                data.append([float(row[1]), float(row[2]), float(row[3])])
            except:
                continue

    data = np.array(data)
    data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-8)
    return data


# ============================================================
# 3. TRAIN + VALIDATION
# ============================================================
def train_lstm():
    data = load_accelerometer_data("accelerometer.csv")

    split = int(0.8 * len(data))
    train_data = data[:split]
    val_data = data[split:]

    model = LSTM(input_size=3, hidden_size=50, learning_rate=0.001)
    epochs = 20

    print("\n--- Training LSTM ---\n")

    for epoch in range(epochs):
        train_loss = 0

        model.h[:] = 0
        model.c[:] = 0

        for t in range(len(train_data) - 1):
            x = train_data[t]
            y_true = train_data[t + 1]

            y_pred = model.forward(x)
            loss = np.mean((y_pred - y_true) ** 2)
            train_loss += loss

            model.backward(y_pred, y_true)

        train_loss /= len(train_data)

        # ----------------------------
        # VALIDATION
        # ----------------------------
        val_loss = 0
        model.h[:] = 0
        model.c[:] = 0

        for t in range(len(val_data) - 1):
            y_pred = model.forward(val_data[t])
            val_loss += np.mean((y_pred - val_data[t + 1]) ** 2)

        val_loss /= len(val_data)

        print(f"Epoch {epoch+1:02d} | Train MSE: {train_loss:.6f} | Val MSE: {val_loss:.6f}")


# ============================================================
# 4. MAIN
# ============================================================
if __name__ == "__main__":
    train_lstm()



--- Training LSTM ---

Epoch 01 | Train MSE: 0.378359 | Val MSE: 0.401785
Epoch 02 | Train MSE: 0.327668 | Val MSE: 0.332391
Epoch 03 | Train MSE: 0.291570 | Val MSE: 0.305895
Epoch 04 | Train MSE: 0.277576 | Val MSE: 0.293964
Epoch 05 | Train MSE: 0.272414 | Val MSE: 0.287569
Epoch 06 | Train MSE: 0.269627 | Val MSE: 0.283831
Epoch 07 | Train MSE: 0.267156 | Val MSE: 0.281605
Epoch 08 | Train MSE: 0.264132 | Val MSE: 0.280177
Epoch 09 | Train MSE: 0.259965 | Val MSE: 0.279137
Epoch 10 | Train MSE: 0.254642 | Val MSE: 0.278652
Epoch 11 | Train MSE: 0.248841 | Val MSE: 0.279565
Epoch 12 | Train MSE: 0.243415 | Val MSE: 0.282555
Epoch 13 | Train MSE: 0.238910 | Val MSE: 0.287118
Epoch 14 | Train MSE: 0.235493 | Val MSE: 0.292534
Epoch 15 | Train MSE: 0.232965 | Val MSE: 0.299421
Epoch 16 | Train MSE: 0.230910 | Val MSE: 0.307244
Epoch 17 | Train MSE: 0.228883 | Val MSE: 0.312302
Epoch 18 | Train MSE: 0.226661 | Val MSE: 0.316111
Epoch 19 | Train MSE: 0.224375 | Val MSE: 0.320946
Epoch 2